In [2]:
%matplotlib inline

In [5]:
import torch
import torch.nn as nn # Use torch.nn as nn
import torch.nn.functional as F # Use torch.nn.functional as F

class Net(nn.Module):
    # Network for Simple Image Processing
    # Spec
    # INPUT : 32 * 32 pixels 1 image
    # Layer 1 : 3 * 3 square kernel -> 28 * 28 pixels with 6 output channels (Convolution) / 14 * 14 pixels with 6 output channels(Subsampling)
    # Layer 2 : 3 * 3 square kernel -> 10 * 10 pixels with 16 output channels (Convolution) / 5 * 5 pixels with 16 output channels(Subsampling)
    # Layer 3 : 120 nodes
    # Layer 4 : 84 nodes
    # Layer 5(OUTPUT) : 10 nodes (labels for 1 to 10)
    def __init__(self):
        super(Net, self).__init__()
        # 2 Convolution Layers
        self.conv1 = nn.Conv2d(1, 6, 3) # 1 input image channel, 6 output channels, 3 x 3 square convolution
        self.conv2 = nn.Conv2d(6, 16, 3) # 6 input channels, 16 output channels, 3 * 3 square convolution
        # 2 Fulley Connected Layers
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        # 1 Output Layer
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv1(x)), 2) # 2 -> equals to (2, 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # no Activation
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
